# Tests historisation


In [1]:
import os
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine
from create_query import to_indicator

engine = create_engine(os.getenv("DATABASE_URL"))

## Tests Solution 1

In [2]:
# fonction qui retourne l'indicateur sous différents formats (par défaut un DataFrame) à partir de la requête définie
to_indicator(engine, 't1-02-75')

,nb_pdc,p_range,code
0,73,"[15.0, 26.0)",75
1,13,"[26, 65.0)",75
2,9,"[0, 15.0)",75
3,4,"[65, 175.0)",75
4,3,"[175, 360.0)",75


In [3]:
# ajout d'indicateurs dans la table 'quotidien'
# on simule l'envoi quotidien des indicateurs (en répétant le même indicateur)
to_indicator(engine, 'i1',          histo=True, format='table', table_name='quotidien', table_option='replace')
to_indicator(engine, 'i1-01-93-02', histo=True, format='table', table_name='quotidien', table_option='append')
to_indicator(engine, 't1-02-75',    histo=True, format='table', table_name='quotidien', table_option='append')
to_indicator(engine, 't3-04-13001', histo=True, format='table', table_name='quotidien', table_option='append')
to_indicator(engine, 'i1',          histo=True, format='table', table_name='quotidien', table_option='append')
to_indicator(engine, 't1-02-75',    histo=True, format='table', table_name='quotidien', table_option='append')
to_indicator(engine, 't3-04-13001', histo=True, format='table', table_name='quotidien', table_option='append')
to_indicator(engine, 't1-02-75',    histo=True, format='table', table_name='quotidien', table_option='append')

,count
0,34


In [4]:
# tous les indicateurs ont le même format
with engine.connect() as conn:
    quotidien = pd.read_sql_table('quotidien', conn)
quotidien

,nombre,somme,crit_v,query,level,val,area,timest
0,0,0,null,null,null,null,null,2024-09-09 23:26:04.565941
1,1,23522,,i1,00,00,00,2024-09-09 23:26:04.550728
2,1,473,13,i1,01,93,02,2024-09-09 23:26:04.694977
3,1,450,06,i1,01,93,02,2024-09-09 23:26:04.694977
4,1,175,83,i1,01,93,02,2024-09-09 23:26:04.694977
5,1,170,84,i1,01,93,02,2024-09-09 23:26:04.694977
6,1,105,04,i1,01,93,02,2024-09-09 23:26:04.694977
7,1,57,05,i1,01,93,02,2024-09-09 23:26:04.694977
8,1,73,"[15.0,26.0)",t1,02,75,00,2024-09-09 23:26:04.753705
9,1,13,"[26,65.0)",t1,02,75,00,2024-09-09 23:26:04.753705


In [5]:
# passage de la table 'quotidien' à la table 'mensuel' (chaque mois on retrouve un seul enregistrement de chaque indicateur)
# le traitement pourrait se faire directement en SQL sans passer par un DataFrame
# la requête est la même pour passer de jour à mois que pour passer de mois à année

query = """
SELECT
  SUM(nombre) AS nombre,  SUM(somme) AS somme,  crit_v,  query,  level,  val,  area
FROM
  quotidien
WHERE
  (timest >= CAST(NOW() AS date))   AND   (timest < CAST((NOW() + INTERVAL '1 month') AS date))
GROUP BY
  crit_v,  query,  level,  val,  area
ORDER BY
  query,  level,  val,  area
"""

with engine.connect() as conn:
    mensuel = pd.read_sql_query(query, conn)

mensuel.to_sql('mensuel', engine, if_exists='replace', index=False)

mensuel

,nombre,somme,crit_v,query,level,val,area
0,2.0,47044.0,,i1,00,00,00
1,1.0,105.0,04,i1,01,93,02
2,1.0,57.0,05,i1,01,93,02
3,1.0,450.0,06,i1,01,93,02
4,1.0,473.0,13,i1,01,93,02
5,1.0,175.0,83,i1,01,93,02
6,1.0,170.0,84,i1,01,93,02
7,0.0,0.0,null,null,null,null,null
8,3.0,27.0,"[0,15.0)",t1,02,75,00
9,3.0,219.0,"[15.0,26.0)",t1,02,75,00


In [6]:
# indicateur 't1-m-02-75' sur le mois
# la requête est générique (en séparant les cas ou c'est la moyenne de ceux ou c'est la somme)

query = """
SELECT
  somme / nombre AS nb_pdc,  crit_v AS p_range
FROM
  mensuel
WHERE
  query = 't1' AND level = '02' AND val = '75'
"""
with engine.connect() as conn:
    t1_m_02_75 = pd.read_sql_query(query, conn)

# on retrouve l'indicateur du jour
t1_m_02_75

,nb_pdc,p_range
0,9.0,"[0,15.0)"
1,73.0,"[15.0,26.0)"
2,3.0,"[175,360.0)"
3,13.0,"[26,65.0)"
4,4.0,"[65,175.0)"
